![title](https://chrisostrouchov.com/images/sqlite_with_python/python_sqlite.png)

# SQL in python 

 Python is very flexible and has a wide range of libraries and third-party modules to support many operations. SQL (Structured Query Language) can be executed from within Python using sqlite3. The sqlite3 module offers support to connect to an external database and execute SQL queries. However, this module does not offer the complete querying capability of a typical SQL engine and functions as a light-weight API version of the querying engine. Other modules like MySQLdb (same as mysql-python), offer a more extensive range of functions and query processing abilities

Python comes with SQLite3, which provides a lightweight disk-based database that doesn't require a seperate server process. It's useful to prototyp with SQLite and then port the code to a larger database system, like MySQL. Python comes with a pretty awesome module to connect to a SQL database with SQLite. The module is SQLite3

In [34]:
#importing sqlite3
import sqlite3

## Connect() method 

The first step in executing SQL through Python is connecting to an external database file. The 'connect' method in the sqlite module helps to create a connection with an external database. The method accepts the name of the external database as argument. We can also create a database in-memory by passing ":memory:" as the argument, however care needs to be taken as this consumes RAM.

The connection is stored as a connection object. Methods like cursor, commit, close, rollback, execute, create_function, etc., can be called on the connection object. In order to learn the full-range of methods and their descriptions, please refer to the sqlite module documentation (Link: https://docs.python.org/2/library/sqlite3.html#module-sqlite3).

To use sqlite3, you must first create a Connection object that represents the database. If the database name already exists SQLite3 will automatically connect to it, if it does not exsist, SQLite3 will automatically create.

For experienced users: You can also supply the special name :memory: to create a database in RAM.

In [6]:
con = sqlite3.connect("pranithmetuku.db")
#or
murcon = sqlite3.connect(':memory:')

## pandas to_sql() - Copying an Existing Data Set¶
 

New tables can be created in a database and data can be inserted through queries, or, data can also be integrated from an existing source table. The 'Murders' data set can be integrated into the murdersdb database using to_sql method of pandas module. There are two steps to do this:

1) Load the data from the source file into a pandas data frame 
2) Use the to_sql method to copy the data from the data frame into a new/existing table 

### Load the murders data from source file into a table called 'murderstable' in the murdersdb database

 

In [7]:
import csv
import pandas as pd

murdersdf = pd.read_csv('https://raw.githubusercontent.com/colaberry/538data/master/murder_2016/murder_2015_final.csv')
murdersdf.head(5)
# renaming columns as SQL columns cannot start with a number
murdersdf.columns = ['city','state','murders_2014','murders_2015','change']
murdersdf.to_sql(name='murderstable',con=murcon,if_exists='replace',index=False)

## Cursor() method 

The cursor method can be called on the connection object. This method directs control for carrying out operations through the connection. Once the cursor object is created, it can be used to carry out querying operations by using methods like execute, executemany, executeall, fetchone, fetchmany, fetchall, rowcount, etc.

1) Execute: SQL queries can be passed as arguments to this method for execution 
2) fetchone/many/all: this method collects the output of the SQL query, one/many/all rows at a time, and prints them out

### Create a cursor object on the connection created previously. Use the cursor object to execute a 'Select' query to show (use fetchall method to store the query result in a variable 'queryone') the first 5 rows of the table created in the previous exercise. Print 'queryone' variable to see the result. 

In [8]:
murcur = murcon.cursor()
murcur.execute("SELECT * FROM murderstable LIMIT 5")
queryone = murcur.fetchall()
print(queryone)


[('Baltimore', 'Maryland', 211, 344, 133), ('Chicago', 'Illinois', 411, 478, 67), ('Houston', 'Texas', 242, 303, 61), ('Cleveland', 'Ohio', 63, 120, 57), ('Washington', 'D.C.', 105, 162, 57)]


There is an alternate way to execute a query using the read_sql_query() method from pandas. The method can be called on a pandas instance and takes two main arguments - the SQL query to be executed on the data set and the connection object which connects the database

### Use the read_sql_query() method on the 'murdersdf' data frame to print the same output as  above



In [9]:
print(pd.read_sql_query("SELECT * FROM murderstable LIMIT 5",murcon))


         city     state  murders_2014  murders_2015  change
0   Baltimore  Maryland           211           344     133
1     Chicago  Illinois           411           478      67
2     Houston     Texas           242           303      61
3   Cleveland      Ohio            63           120      57
4  Washington      D.C.           105           162      57


### CRUD Operations


Create, read, update and delete opearations performed on databases are oftened referred to as CRUD operations. For details on framing SQL queries for these operations please refer to any online SQL tutorial.

The aim of the following set of exercises is to familiarize us with CRUD operations performed through python. We will the murders data set when needed

### Create table 

Create a table with table name as 'murderstabletwo' and the same columns as the murderstable

Remember: Use triple quotes to enclose arguments which span multiple-lines

In [10]:
try:
    murcur.execute("""CREATE TABLE murderstabletwo (
                      city TEXT,
                      state TEXT,
                      murders_2014 INTEGER,
                      murders_2015 INTEGER,
                      change INTEGER)""")
except Exception as e:
    print(e)

### Reading a table

### Execute a read query, which reads the contents of the murderstable and sorts the output rows in a descending order of 2015 murders. Store the first five rows of this output in a variable ('topfive') and print it out.

In [11]:
murcur.execute("SELECT * FROM murderstable ORDER BY murders_2015 DESC LIMIT 5")
topfive = murcur.fetchall()
print(topfive)

[('Chicago', 'Illinois', 411, 478, 67), ('New York', 'New York', 333, 352, 19), ('Baltimore', 'Maryland', 211, 344, 133), ('Houston', 'Texas', 242, 303, 61), ('Detroit', 'Michigan', 298, 295, -3)]


## Inserting into a table

### Insert the top five rows of data in the 'topfive' list, into the second table ('murderstabletwo') created previously. Use the 'executemany' function in order to insert multiple records using a single query. Retrieve the contents of the 'murderstabletwo' using a SELECT query and store the output into a variable 'querytwo'. Print out 'querytwo'.

In [12]:
murcur.executemany("INSERT INTO murderstabletwo VALUES (?,?,?,?,?)",topfive)
murcur.execute("SELECT * FROM murderstabletwo")
querytwo = murcur.fetchall()
print(querytwo)

[('Chicago', 'Illinois', 411, 478, 67), ('New York', 'New York', 333, 352, 19), ('Baltimore', 'Maryland', 211, 344, 133), ('Houston', 'Texas', 242, 303, 61), ('Detroit', 'Michigan', 298, 295, -3)]


# Updating a row in the table

### Make 'Chicago' the safest city. Update the 2015 murders value for Chicago as zero. Retreive the updated contents of 'murderstabletwo' and store them in a variable 'querythree'. Print 'querythree'

In [13]:
murcur.execute("UPDATE murderstabletwo SET murders_2015=0 WHERE city='Chicago'")
murcur.execute("SELECT * FROM murderstabletwo")
querythree = murcur.fetchall()
print(querythree)

[('Chicago', 'Illinois', 411, 0, 67), ('New York', 'New York', 333, 352, 19), ('Baltimore', 'Maryland', 211, 344, 133), ('Houston', 'Texas', 242, 303, 61), ('Detroit', 'Michigan', 298, 295, -3)]


## Deleting Rows and Dropping Tables 

Deleting rows is the same as Update query. As in SQL, deleting only deletes the data and does not delete the table itself. 'Drop' is the command that can be used to delete a table

### Delete the table 'murderstabletwo' and 'murderstable'

 

In [15]:
try:
    murcur.execute("DROP TABLE murderstable")
    murcur.execute("DROP TABLE murderstabletwo")
except Exception as e:
    print(e)

## Commit, rollback and close¶
 

The operations performed to the database can all be saved by calling the commit method on the connection object. Another method 'rollback' can rollback all the changes done to the database after the last commit. 'Close' method called on the connection object closes the connection to the database

## Rollback changes made to the database. Commit the changes and close the connection.



In [17]:
try:
    murcon.rollback()
    murcon.commit()
    murcon.close()
except Exception as e:
    print(e)

# EXample for sqllite3 in python

In [28]:
class Employee:
    """A sample Employee class"""

    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.pay = pay

    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)

    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)

    def __repr__(self):
        return "Employee('{}', '{}', {})".format(self.first, self.last, self.pay)



In [33]:
import sqlite3
conn = sqlite3.connect(':memory:')

c = conn.cursor()

c.execute("""CREATE TABLE employees (
            first text,
            last text,
            pay integer
            )""")


def insert_emp(emp):
    with conn:
        c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp.first, 'last': emp.last, 'pay': emp.pay})


def get_emps_by_name(lastname):
    c.execute("SELECT * FROM employees WHERE last=:last", {'last': lastname})
    return c.fetchall()


def update_pay(emp, pay):
    with conn:
        c.execute("""UPDATE employees SET pay = :pay
                    WHERE first = :first AND last = :last""",
                  {'first': emp.first, 'last': emp.last, 'pay': pay})


def remove_emp(emp):
    with conn:
        c.execute("DELETE from employees WHERE first = :first AND last = :last",
                  {'first': emp.first, 'last': emp.last})

emp_1 = Employee('John', 'Doe', 80000)
emp_2 = Employee('Jane', 'Doe', 90000)

insert_emp(emp_1)
insert_emp(emp_2)

emps = get_emps_by_name('Doe')
print(emps)

update_pay(emp_2, 95000)
remove_emp(emp_1)

emps = get_emps_by_name('Doe')
print(emps)

conn.close()

[('John', 'Doe', 80000), ('Jane', 'Doe', 90000)]
[('Jane', 'Doe', 95000)]


## EXample-2 sqlite3 in python 

In [35]:
import sqlite3

conn = sqlite3.connect('prnith')

c = conn.cursor()

c.execute("""drop table if exists towns""")
c.execute("""drop table if exists hotels""")

conn.commit()

c.execute("""create table towns (
        tid     int     primary key not NULL ,
        name    text,
        postcode        text)""")

c.execute("""create table hotels (
        hid     int     primary key not NULL ,
        tid     int,
        name    text,
        address text,
        rooms   int,
        rate    float)""")

c.execute("""insert into towns values (1, "Melksham", "SN12")""")
c.execute("""insert into towns values (2, "Cambridge", "CB1")""")
c.execute("""insert into towns values (3, "Foxkilo", "CB22")""")

c.execute("""insert into hotels values (1, 2, "Hamilkilo Hotel", "Chesterton Road", 15, 40.)""")
c.execute("""insert into hotels values (2, 2, "Arun Dell", "Chesterton Road", 60, 70.)""")
c.execute("""insert into hotels values (3, 2, "Crown Plaza", "Downing Street", 100, 105.)""")
c.execute("""insert into hotels values (4, 1, "Well House Manor", "Spa Road", 5, 80.)""")
c.execute("""insert into hotels values (5, 1, "Beechfield House", "The Main Road", 26, 110.)""")

conn.commit()

c.execute ("""select * from towns left join hotels on towns.tid = hotels.tid""")

for row in c:
        print (row)

c.close()

(1, 'Melksham', 'SN12', 4, 1, 'Well House Manor', 'Spa Road', 5, 80.0)
(1, 'Melksham', 'SN12', 5, 1, 'Beechfield House', 'The Main Road', 26, 110.0)
(2, 'Cambridge', 'CB1', 1, 2, 'Hamilkilo Hotel', 'Chesterton Road', 15, 40.0)
(2, 'Cambridge', 'CB1', 2, 2, 'Arun Dell', 'Chesterton Road', 60, 70.0)
(2, 'Cambridge', 'CB1', 3, 2, 'Crown Plaza', 'Downing Street', 100, 105.0)
(3, 'Foxkilo', 'CB22', None, None, None, None, None, None)


# Example-3 sqlite in python 

In [36]:
import sqlite3
connection = sqlite3.connect("company.db")

cursor = connection.cursor()

# delete 
#cursor.execute("""DROP TABLE employee;""")

sql_command = """
CREATE TABLE employee ( 
staff_number INTEGER PRIMARY KEY, 
fname VARCHAR(20), 
lname VARCHAR(30), 
gender CHAR(1), 
joining DATE,
birth_date DATE);"""

cursor.execute(sql_command)

sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (NULL, "William", "Shakespeare", "m", "1961-10-25");"""
cursor.execute(sql_command)


sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (NULL, "Frank", "Schiller", "m", "1955-08-17");"""
cursor.execute(sql_command)

# never forget this, if you want the changes to be saved:
connection.commit()

connection.close()

In [39]:
import sqlite3
connection = sqlite3.connect("company.db")

cursor = connection.cursor()

cursor.execute("SELECT * FROM employee") 
print("fetchall:")
result = cursor.fetchall() 
for r in result:
    print(r)
cursor.execute("SELECT * FROM employee") 
print("\nfetch one:")
res = cursor.fetchone() 
print(res)

fetchall:
(1, 'William', 'Shakespeare', 'm', None, '1961-10-25')
(2, 'Frank', 'Schiller', 'm', None, '1955-08-17')

fetch one:
(1, 'William', 'Shakespeare', 'm', None, '1961-10-25')


# Additional information about sqlite 

https://www.pythoncentral.io/introduction-to-sqlite-in-python/

https://github.com/brownan/SQLite-tutorial/blob/master/SQLite_Tutorial.ipynb